# Summarization seminar

## Resources 

* English data CNN/Daily https://cs.nyu.edu/~kcho/DMQA/ http://nlpprogress.com/english/summarization.html
* Article. Dataset for Automatic Summarization of Russian News: https://arxiv.org/abs/2006.11063
* Download Russian dataset: https://github.com/IlyaGusev/gazeta
* Pretrained Mbart Summarization models: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta 
https://github.com/IlyaGusev/summarus

## Requirements

In [ ]:
!pip install --upgrade transformers razdel torch fasttext tensorboardX nltk tensorflow pymorphy2[fast] rouge==0.3.1 summa sumy lexrank

Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)
Requirement already up-to-date: razdel in /usr/local/lib/python3.7/dist-packages (0.5.0)
Requirement already up-to-date: torch in /usr/local/lib/python3.7/dist-packages (1.8.1+cu101)
Requirement already up-to-date: fasttext in /usr/local/lib/python3.7/dist-packages (0.9.2)
Requirement already up-to-date: tensorboardX in /usr/local/lib/python3.7/dist-packages (2.2)
Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.5)
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)
Requirement already up-to-date: pymorphy2[fast] in /usr/local/lib/python3.7/dist-packages (0.9.1)
  Using cached https://files.pythonhosted.org/packages/8f/89/af359c22e1d858e0299d4cc9219f36b504817c9797acad23081247867845/rouge-0.3.1-py3-none-any.whl
Requirement already up-to-date: summa in /usr/local/lib/python3.7/dist-packages (1.2.0)
Requirement already up-

In [ ]:
!git clone https://github.com/IlyaGusev/summarus
%cd summarus
!git checkout 687bd6f
!pip install -r requirements.txt
%matplotlib inline

fatal: destination path 'summarus' already exists and is not an empty directory.
/content/summarus
HEAD is now at 687bd6f Better oracle builder
  Using cached https://files.pythonhosted.org/packages/63/ac/b93411318529980ab7f41e59ed64ec3ffed08ead32389e29eb78585dd55d/rouge-0.3.2-py3-none-any.whl
     |████████████████████████████████| 153kB 19.5MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: rouge 0.3.1
    Uninstalling rouge-0.3.1:
      Successfully uninstalled rouge-0.3.1
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


## Data

#### Russian data

In [ ]:
!rm -f gazeta_jsonl.tar.gz
!wget -nc https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
!tar -xzvf gazeta_jsonl.tar.gz

--2021-04-05 20:49:29--  https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz [following]
--2021-04-05 20:49:29--  https://www.dropbox.com/s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucebcf57ff30e0a667faf08933bc.dl.dropboxusercontent.com/cd/0/inline/BMG14xRykZFc97FvZ9tddItYtnnaqC2AjDWZy-aEBHGZVET0O9uQfQlQLc0liMDTjDk9fotuUn6VBPotpd0pyIjMb50JOc2tvVsuHO-E-laTc2gwRzSCDYfSQXdzgJ_8MEUEA_U2JDhvwzcKeAOWNWTx/file# [following]
--2021-04-05 20:49:30--  https://ucebcf57ff30e0a667faf08933bc.dl.dropboxusercontent.com/cd/0/inline/BMG14xRykZFc97FvZ9tddItYtnnaqC2AjDWZy-aEBHGZVET0O9uQfQlQLc0liMDTjDk9f

In [ ]:
import json
import random

def read_gazeta_records(file_name, shuffle=False, sort_by_date=True):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle(records)
    return records

train_records = read_gazeta_records("gazeta_train.jsonl")
val_records = read_gazeta_records("gazeta_val.jsonl")
test_records = read_gazeta_records("gazeta_test.jsonl")

In [ ]:
test_records[0]

{'date': '2019-10-01 08:23:02',
 'summary': 'Октябрь богат на изменения в законодательстве. В бюджетной сфере с 1 октября повысят зарплаты. Заемщикам снизят показатель предельной долговой нагрузки, при которой возможна выдача кредита. Запретят курить на балконах и в отелях. И введут 20%-й налог на пальмовое масло, чтобы кондитеры меньше травили россиян.',
 'text': 'В октябре 2019 года в России вступают в силу ряд новаций, которые существенно затрагивают интересы россиян. Итак, с 1 числа ожидается повышение зарплат работникам бюджетной сферы. Тех бюджетников, которые не затронуты майскими указами президента. Повышение коснется персонала детских садов (нянечек), юристов, бухгалтеров, технических работников и социальных психологов. Прибавку получат также военные, сотрудники органов внутренних дел , уголовно-исполнительной системы наказаний, служащие национальной гвардии. У работников учреждений, не относящихся к социальной сфере и науке, зарплаты будут повышены на 3,8%. Эта новация затрон

#### English data

In [ ]:
!tar xvf /content/cnn_stories.tgz
!ls -ltr | wc -l

Выходные данные были обрезаны до нескольких последних строк (5000).
./cnn/stories/28ca66b78f32b0395bcf89658121708c12d2adaa.story
./cnn/stories/4ccc93faef1eeac4c661a4369edbf324bd81202b.story
./cnn/stories/2bba1b8102456ebcdee37b375cca7097956eff1d.story
./cnn/stories/6beafefada5649c04532798b97f57ce64691ef9b.story
./cnn/stories/18565de60a3da497f26c7998339cfa29817ae305.story
./cnn/stories/59bcc9136dec19cbac2d6bd4d6d13e3caaeb40cb.story
./cnn/stories/45409369da2dc0b467417b4623c26d46c0aa0944.story
./cnn/stories/22edd27f817234833c5cff3cc55bd278002dffba.story
./cnn/stories/8e20fca3d8c15ee2ea9c706d3130213198cf06af.story
./cnn/stories/1122704b29b7cc3034fdf40aae78070b5ec7fd79.story
./cnn/stories/0587309e69611037f4293a898fbb57f2761b6347.story
./cnn/stories/e0127880a6d047cf7adf5b8ceca22873c7b4c86b.story
./cnn/stories/a9a0d4af0c7ab1750911e3244afd881be730ce77.story
./cnn/stories/2718cfa01c16a5360fb4898aa27b010bafad35b7.story
./cnn/stories/dfeb9d89eda7649370f912129dd73be7cfb7fd77.story
./cnn/stories/d39

In [ ]:
!head cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story

(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?

Some scoff at the notion that movies do anything more than entertain. They are wrong. Sure, it's unlikely that one movie alone will change your views on issues of magnitude. But a movie (or TV show) can begin your "education" or "miseducation" on a topic. And for those already agreeing with the film's thesis, it can further entrench your views.

Anyone who doubts the potential influence that movies can have on public opinion need to look no further than two films that are causing an uproar even before they have opened nationwide. They present hot button issues that manage to fire up people from the left and right.

The first, "Zero Dark Thirty," is about the pursuit and killing of Osama bin Laden, which features scenes 

In [ ]:
import os

def load_doc(filename):
  with open(filename, encoding='utf-8') as inf:
	  	return inf.read()

def split_story(doc):
	index = doc.find('@highlight')
	# split into story and highlights
	story, highlights = doc[:index], doc[index:].split('@highlight')
	highlights = [h.strip() for h in highlights if len(h) > 0]
	return story, highlights

# load all stories in a directory
def load_stories(directory):
	stories = list()
	for name in os.listdir(directory):
		filename = directory + '/' + name
		doc = load_doc(filename)
		story, highlights = split_story(doc)
		stories.append({'story':story, 'highlights':highlights})
	return stories

# load stories
directory = 'cnn/stories/'
stories = load_stories(directory)
print('Loaded Stories %d' % len(stories))
print(stories[4]['story'])
print(stories[4]['highlights'])

Loaded Stories 92579
(CNN) -- A film poet and provocateur probably more famous for his bombed-out appearances on David Letterman, Harmony Korine made a name for himself in 1995 with "Kids," Larry Clark's brutally unsentimental film about the teenage sex drive from a screenplay Korine had written when he was 18.

Fast-forward nearly two decades. Korine, now 40, is no longer the enfant terrible. He's married, a father, and perhaps starting to chafe against the marginalization that comes with such willfully independent ventures as "Gummo," "Julian Donkey-Boy" and "Mister Lonely." His last film, "Trash Humpers," was a fake found-footage romp about the exploits of senior citizens, hooligans who roamed the slums at night causing mayhem. It appeared on several critics' best and worst films of the year lists but scarcely caused a blip on the mainstream cultural radar.

Korine's new film, "Spring Breakers," has already made a bigger splash and represents a rare potential intersection between th

In [ ]:
from sklearn.model_selection import train_test_split
import random
import string


def clean_lines(lines):
  cleaned = list()
  table = str.maketrans('', '', string.punctuation)
  for line in lines:
    index = line.find('(CNN) -- ')
    if index > -1:
      line = line[index+len('(CNN)'):]
    line = line.split()
    line = [word.lower() for word in line]
    line = [w.translate(table) for w in line]
    line = [word for word in line if word.isalpha()]
    cleaned.append(' '.join(line))
  cleaned = ". ".join([c for c in cleaned if len(c) > 0])
  return cleaned

english_data = []
for example in stories:
    story = clean_lines(example['story'].split('\n'))
    highlights = clean_lines(example['highlights'])
    if highlights != "":
      row = {"text": story, "summary": highlights}
      english_data.append(row)

shuffle_data = random.sample(english_data, len(english_data))
train_records_en, rest = train_test_split(shuffle_data, test_size=0.4, random_state=42)
val_records_en, test_records_en = train_test_split(rest, test_size=0.4, random_state=42)
len(train_records_en), len(val_records_en), len(test_records_en)

(55547, 22219, 14813)

In [ ]:
test_records_en[1]

{'summary': 'tribunal convicts seven bosnian serbs over srebrenica massacre. men convicted of genocide extermination murder and other charges. muslim boys and men were killed in the massacre. killing was worst european massacre since world war ii',
 'text': 'seven former highranking bosnian serb military and police officials were convicted thursday for their roles in the notorious srebrenica massacre the war crimes tribunal in the hague said. the men were found guilty of a range of crimes including genocide extermination murder and persecution according to the international criminal tribunal for the former yugoslavia icty. the trial was the largest one to date held at the tribunal the icty said. more than muslim men and boys were killed in the bosnian town of srebrenica in when ethnic serb troops overran a united nations safe area the fiveday slaughter was the worst european massacre since world war ii and was described by the un war crimes tribunal as the triumph of evil. the serbian 

## Metrics

In [ ]:
!rm -f meteor-1.5.tar.gz
!wget -nc https://www.cs.cmu.edu/~alavie/METEOR/download/meteor-1.5.tar.gz
!tar -xzvf meteor-1.5.tar.gz

--2021-04-05 18:32:57--  https://www.cs.cmu.edu/~alavie/METEOR/download/meteor-1.5.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223646468 (213M) [application/x-gzip]
Saving to: ‘meteor-1.5.tar.gz’

meteor-1.5.tar.gz   100%[===================>] 213.29M  2.99MB/s    in 52s     

2021-04-05 18:33:49 (4.07 MB/s) - ‘meteor-1.5.tar.gz’ saved [223646468/223646468]

meteor-1.5/
meteor-1.5/data/
meteor-1.5/data/paraphrase-es.gz
meteor-1.5/data/paraphrase-cz.gz
meteor-1.5/data/paraphrase-fr.gz
meteor-1.5/data/paraphrase-en.gz
meteor-1.5/data/paraphrase-de.gz
meteor-1.5/data/paraphrase-ru.gz
meteor-1.5/xray/
meteor-1.5/xray/visualize_alignments.py
meteor-1.5/xray/template/
meteor-1.5/xray/template/score.tex
meteor-1.5/xray/xray.py
meteor-1.5/xray/MeteorAlignment.py
meteor-1.5/xray/Generation.py
meteor-1.5/scripts/
meteor-1.5/scripts/sgmlize.py
met

In [ ]:
from summarus.util.metrics import print_metrics
from evaluate import postprocess

def calc_method_score(records, predict_func, language="ru", nrows=None):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if nrows is not None and i >= nrows:
            break
        summary = record["summary"]
        text = record["text"]
        prediction = predict_func(text, summary)
        references.append(summary)
        predictions.append(prediction)
    
    for i, (ref, hyp) in enumerate(zip(references, predictions)):
        references[i], predictions[i] = postprocess(ref, hyp, language=language, tokenize_after=True, lower=True)
    print_metrics(references, predictions, language=language, meteor_jar="meteor-1.5/meteor-1.5.jar")

In [ ]:
refs = [
  [
    "this is one reference sentence for sentence1",
    "this is a reference sentence for sentence2 which was generated by your model"
  ],
  [
    "this is one more reference sentence for sentence1",
    "this is the second reference sentence for sentence2"
  ]
]

hyps = [
    "this is the model generated sentence1 which seems good enough",
    "this is sentence2 which has been generated by your model"
]

In [ ]:
!pip install rouge
from rouge import Rouge  
rouge = Rouge()
scores = rouge.get_scores(hyps, [ref[0] for ref in refs], avg=True)
scores

{'rouge-1': {'f': 0.28758169445512416, 'p': 0.25, 'r': 0.3392857142857143},
 'rouge-2': {'f': 0.12916666180572933,
  'p': 0.1111111111111111,
  'r': 0.15476190476190477},
 'rouge-l': {'f': 0.274884074585556, 'p': 0.25, 'r': 0.3392857142857143}}

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

corpus_bleu(refs, hyps)

0.476193974481644

In [ ]:
# !pip install git+https://github.com/Maluuba/nlg-eval.git@master
# !nlg-eval --setup
from nlgeval import NLGEval

nlgeval = NLGEval()  # loads the models
metrics_dict = nlgeval.compute_metrics(refs, hyps)
metrics_dict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Installing to /root/.cache/nlgeval
In case of incomplete downloads, delete the directory and run `nlg-eval --setup /root/.cache/nlgeval' again.
glove2word2vec.py: 100% 1.00/1.00 [00:00<00:00, 346 chunks/s]
dictionary.txt: 550 chunks [00:00, 944 chunks/s]
uni_skip.npz: 100% 634/634 [00:19<00:00, 33.1 chunks/s]
uni_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 1.68k chunks/s]
bi_skip.npz: 100% 276/276 [00:28<00:00, 9.69 chunks/s]
bi_skip.npz.pkl: 100% 1.00/1.00 [00:00<00:00, 1.86k chunks/s]
multi-bleu.perl: 100% 1.00/1.00 [00:00<00:00, 752 chunks/s]
utable.npy: 100% 2.23k/2.23k [02:06<00:00, 17.6 chunks/s]
btable.npy: 100% 2.23k/2.23k [02:09<00:00, 17.2 chunks/s]
glove.6B.zip: 100% 823/823 [02:40<00:00, 5.14 chunks/s]
2021-04-05 19:42:36,951 : MainThread : INFO : 400000 lines with 300 dimensions
2021-04-05 19:42:50,323 : MainThread : INFO : Model /root/.cache/nlgeval/glove.6B.300d.

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


{'Bleu_1': 0.5499999999725,
 'Bleu_2': 0.4281744192662396,
 'Bleu_3': 0.28404282012721094,
 'Bleu_4': 0.20114343061802883,
 'CIDEr': 1.2421919556558583,
 'EmbeddingAverageCosineSimilairty': 0.88469,
 'EmbeddingAverageCosineSimilarity': 0.88469,
 'GreedyMatchingScore': 0.784205,
 'METEOR': 0.2957971080379661,
 'ROUGE_L': 0.5221037854154413,
 'SkipThoughtCS': 0.6261495,
 'VectorExtremaCosineSimilarity': 0.568696}

## Lead

First sentences of a text as a baseline.

Lead-3 (first three sentences) was a long time a great baseline. 

In [ ]:
import razdel

def predict_lead(text, summary, n):
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    prediction = " ".join(sentences[:n])
    return prediction

def en_lead(text, summary, n):
    return text[:n]

In [ ]:
calc_method_score(test_records, lambda x, y: predict_lead(x, y, 1))

-------------METRICS-------------
Count:	 5770
Ref:	 канцлер германии ангела меркель , будучи на домашнем карантине , сдала первый тест на коронавирус , который дал отрицательный результат . 65-летняя политик решила самоизолироваться после того , как оказалось , что у врача , с которым она контактировала , обнаружили инфекцию . в настоящее время она продолжает выполнять свои обязанности по видеосвязи .
Hyp:	 канцлер германии ангела меркель сдала первый тест на наличие коронавируса .
BLEU:     	19.9
ROUGE-1-F:	27.6
ROUGE-2-F:	12.9
ROUGE-L-F:	20.2
Dup 1-grams:	6.0
Dup 2-grams:	0.1
Dup 3-grams:	0.0


In [ ]:
calc_method_score(test_records, lambda x, y: predict_lead(x, y, 2))

-------------METRICS-------------
Count:	 5770
Ref:	 канцлер германии ангела меркель , будучи на домашнем карантине , сдала первый тест на коронавирус , который дал отрицательный результат . 65-летняя политик решила самоизолироваться после того , как оказалось , что у врача , с которым она контактировала , обнаружили инфекцию . в настоящее время она продолжает выполнять свои обязанности по видеосвязи .
Hyp:	 канцлер германии ангела меркель сдала первый тест на наличие коронавируса . его результат отрицательный , сообщил агентству dpa официальный представитель кабинета министров фрг штеффен зайберт .
BLEU:     	43.1
ROUGE-1-F:	30.6
ROUGE-2-F:	13.7
ROUGE-L-F:	25.6
Dup 1-grams:	13.3
Dup 2-grams:	0.8
Dup 3-grams:	0.2


In [ ]:
calc_method_score(test_records, lambda x, y: predict_lead(x, y, 3))

-------------METRICS-------------
Count:	 5770
Ref:	 канцлер германии ангела меркель , будучи на домашнем карантине , сдала первый тест на коронавирус , который дал отрицательный результат . 65-летняя политик решила самоизолироваться после того , как оказалось , что у врача , с которым она контактировала , обнаружили инфекцию . в настоящее время она продолжает выполнять свои обязанности по видеосвязи .
Hyp:	 канцлер германии ангела меркель сдала первый тест на наличие коронавируса . его результат отрицательный , сообщил агентству dpa официальный представитель кабинета министров фрг штеффен зайберт . « результат теста , который был сделан сегодня , отрицательный » , — отметил он .
BLEU:     	44.2
ROUGE-1-F:	31.0
ROUGE-2-F:	13.4
ROUGE-L-F:	26.3
Dup 1-grams:	18.4
Dup 2-grams:	1.5
Dup 3-grams:	0.4


##TextRank

TextRank – is a graph-based ranking model for text processing which can be used in order to find the most relevant sentences in text and also to find keywords.

**Identify relevant sentences**
In order to find the most relevant sentences in text, a graph is constructed the way so the vertices of the graph represent each sentence in a document and the edges between sentences are based on content overlap, namely by *calculating the number of words* that 2 sentences have in common.

Based on this network of sentences, the sentences are fed into the Pagerank algorithm which identifies the most important sentences. When we want to extract a summary of the text, we can now take only the most important sentences.


* Original paper: https://www.aclweb.org/anthology/W04-3252.pdf
* Summa library: https://github.com/summanlp/textrank


Implemented in Gensim summarizer is actually a TextRank:

https://www.machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/


In [ ]:
from summa.summarizer import summarize

def predict_text_rank(text, summary, summary_part=0.1):
    return summarize(text, ratio=summary_part, language='russian').replace("\n", " ")

calc_method_score(test_records, predict_text_rank)

-------------METRICS-------------
Count:	 5770
Ref:	 канцлер германии ангела меркель , будучи на домашнем карантине , сдала первый тест на коронавирус , который дал отрицательный результат . 65-летняя политик решила самоизолироваться после того , как оказалось , что у врача , с которым она контактировала , обнаружили инфекцию . в настоящее время она продолжает выполнять свои обязанности по видеосвязи .
Hyp:	 « канцлер чувствует себя хорошо , она подключилась сегодня по телефону и провела заседание правительства , она выполняет свою работу из дома » , — сообщил официальный представитель кабмина фрг , отметив , что канцлер следует всем рекомендациям врачей . « канцлер на сегодняшний момент здорова , она сидит в домашнем офисе , как это положено всем , кто отправился в домашнюю самоизоляцию . тогда же он отметил , что возможно сдаст еще один тест , так как не исключает заражения во время работы в последнее время .
BLEU:     	28.5
ROUGE-1-F:	21.4
ROUGE-2-F:	6.3
ROUGE-L-F:	16.4
Dup 1-grams:

In [ ]:
import pprint
from gensim.summarization import summarize
from gensim.summarization import keywords

pprint.pprint(test_records[1]["text"])
print(summarize(test_records[1]["text"]))

('С 1 октября 2019 года любой кредитор — банк, микрофинансовая организация, '
 'потребительский или сельскохозяйственный кооператив — принимая решение о '
 'выдаче человеку (или его семье) нового кредита, должен учитывать уже '
 'сложившийся на момент запроса заемных денег показатель долговой нагрузки '
 'заемщика (ПДН). Банки и МФО будут рассчитывать ПДН по кредитам от 10 тысяч '
 'рублей как отношение ежемесячных платежей по кредиту к ежемесячному доходу '
 'заемщика. Но никакого норматива, соблюдать который кредиторы должны начать '
 '«прямо сегодня», не вводится. Банк России объявил о своем намерении следить '
 'за показателем долговой нагрузки по отношению к доходам еще в августе 2018 '
 'года и пока собирается просто наблюдать, какое соотношения дохода к величине '
 'выплат по новому кредиту сочтут для себя комфортным банки и МФО, чтобы и '
 'самим быть в прибыли от кредитования, и чтобы одновременно заемщик мог '
 'оплачивать кредит (или кредиты), не рискуя «пойти по миру» или, 

In [ ]:
from gensim.summarization import summarize
from gensim.summarization import keywords

print("Origin: ", test_records_en[1]["text"])
print(keywords(test_records_en[1]["text"]))
print("==========")
#we want 50% of the original text (the default is 20%).
print(summarize(test_records_en[1]["text"], ratio=0.5))
print("========")
# specify max word count 
print(summarize(test_records_en[1]["text"], word_count=50))

Origin:  cnn lewis hamilton will take time out from his preparations for the malaysia grand prix to see his disabled halfbrother follow in his footsteps and compete on the racing circuit for the first time. the formula one world champion will make the trip from kuala lumpur to britain to see nicolas who has cerebral palsy make his racing debut at the renault clio cup at brands hatch on april. the son of anthony hamilton and his second wife linda has suffered from the condition all his life and is unable to stand or walk for long periods of time however he is able to race alongside his ablebodied competitors courtesy of a specially modified car. mclaren driver hamilton has said in the past his younger sibling is his biggest inspiration and confirmed to reporters on tuesday that he would be trackside to support nicolas in his first race. i am the closest person to him but even i cannot comprehend what it must have been like to have the difficulties he has had the briton told reporters. i

## Lexrank
* Original paper: https://arxiv.org/pdf/1109.2128.pdf
* lexrank library: https://github.com/crabcamp/lexrank

A sentence which is similar to many other sentences of the text has a high probability of being important. The approach of LexRank is that a particular sentence is recommended by other similar sentences and hence is ranked higher. It uses cosine similarity of TF-IDF vectors. LexRank was used as part of a larger summarization system (MEAD) that combines the LexRank score (stationary probability) with other features like sentence position and length using a linear combination with either user-specified or automatically tuned weights.

In [ ]:
import lexrank
from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS

def predict_lex_rank(text, summary, lxr, summary_size=3, threshold=None):
    sentences = [s.text for s in razdel.sentenize(text)]
    prediction = lxr.get_summary(sentences, summary_size=summary_size, threshold=threshold)
    prediction = " ".join(prediction)
    return prediction

sentences = [[s.text for s in razdel.sentenize(r["text"])] for r in test_records]
lxr = LexRank(sentences, stopwords=STOPWORDS['ru'])

In [ ]:
calc_method_score(test_records, lambda x, y: predict_lex_rank(x, y, lxr))

-------------METRICS-------------
Count:	 5770
Ref:	 канцлер германии ангела меркель , будучи на домашнем карантине , сдала первый тест на коронавирус , который дал отрицательный результат . 65-летняя политик решила самоизолироваться после того , как оказалось , что у врача , с которым она контактировала , обнаружили инфекцию . в настоящее время она продолжает выполнять свои обязанности по видеосвязи .
Hyp:	 канцлер германии ангела меркель сдала первый тест на наличие коронавируса . после того как стало известно о первых зараженных из делегации , трамп сдал тест , который дал отрицательный результат . по его словам , как первичный тест , так и вторичный дали отрицательный результат .
BLEU:     	37.7
ROUGE-1-F:	23.7
ROUGE-2-F:	7.8
ROUGE-L-F:	19.9
Dup 1-grams:	24.0
Dup 2-grams:	4.5
Dup 3-grams:	1.7


## LSA

Use sentence vectors instead documents. Find again the clusters of most valuable.

* One of the original papers: https://www.cs.bham.ac.uk/~pxt/IDA/text_summary.pdf
* sumy library: https://github.com/miso-belica/sumy

In [ ]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
import nltk; nltk.download('punkt');

def predict_lsa(text, summary, lsa_summarizer, tokenizer, summary_size=3):
    parser = PlaintextParser.from_string(text, tokenizer)
    predicted_summary = lsa_summarizer(parser.document, summary_size)
    predicted_summary = " ".join([str(s) for s in predicted_summary])
    return predicted_summary

lsa_summarizer = LsaSummarizer()
tokenizer = Tokenizer("russian")
calc_method_score(test_records, lambda x, y: predict_lsa(x, y, lsa_summarizer, tokenizer))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


-------------METRICS-------------
Count:	 5770
Ref:	 канцлер германии ангела меркель , будучи на домашнем карантине , сдала первый тест на коронавирус , который дал отрицательный результат . 65-летняя политик решила самоизолироваться после того , как оказалось , что у врача , с которым она контактировала , обнаружили инфекцию . в настоящее время она продолжает выполнять свои обязанности по видеосвязи .
Hyp:	 несколько дней назад стало известно , что положительный результат на коронавирус дал тест главы кабинета институциональной безопасности канцелярии бразильского президента аугусту элено . таким образом , он стал шестнадцатым членом делегации политиков из бразилии , которые участвовали во встрече с американским лидером дональдом трампом и у которых выявлен коронавирус . пройти анализы он решил после того , как у главы секретариата по общественным коммуникациям при его администрации фабио вайнгартена обнаружили это заболевание .
BLEU:     	30.7
ROUGE-1-F:	19.3
ROUGE-2-F:	5.0
ROUGE-L-F

## SummaRuNNer

SummaRuNNer: A RNN based Sequence Model for Extractive Summarization of Documents
* Original paper: https://arxiv.org/abs/1611.04230

SummaRuNNer: A two-layer bidirectional RNN (GRU) based sequence classifier: the bottom layer operates at word level within each sentence, while the top layer runs over sentences. The top layer with 1’s and 0’s is the sigmoid activation based classification layer that decides whether
or not each sentence belongs to the summary. The decision at
each sentence depends on the content richness of the sentence, its
salience with respect to the document, its novelty with respect to
the accumulated summary representation and other positional features.

In [ ]:
!wget https://www.dropbox.com/s/mlo7ioxodqib1xl/gazeta_summarunner_3kk.tar.gz

--2020-09-15 09:41:14--  https://www.dropbox.com/s/mlo7ioxodqib1xl/gazeta_summarunner_3kk.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/mlo7ioxodqib1xl/gazeta_summarunner_3kk.tar.gz [following]
--2020-09-15 09:41:14--  https://www.dropbox.com/s/raw/mlo7ioxodqib1xl/gazeta_summarunner_3kk.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9e8c908e8a14081236cc3f1ce4.dl.dropboxusercontent.com/cd/0/inline/A_bUhE2SFODanQ9WdQTS_j5gbVNO_KUQfr-CpzKDlsbIu8EoqEQCIb_jb3RhOsrk7JHMnC_KelrrpWOjqhaUivzvCNYVcYC9DuZoiN93nwUodyiZdUmMqVKC3H_8TkBHu8c/file# [following]
--2020-09-15 09:41:14--  https://uc9e8c908e8a14081236cc3f1ce4.dl.dropboxusercontent.com/cd/0/inline/A_bUhE2SFODanQ9WdQTS_j5gbVNO_KUQfr-CpzKDlsbIu8EoqEQCIb_jb3RhOsr

In [ ]:
!bash predict.sh -t gazeta_test.jsonl -m gazeta_summarunner_3kk.tar.gz -p subwords_summary_sentences \
    -L ru -T -D -P summarunner_predictions.txt -M meteor-1.5/meteor-1.5.jar

Calling AllenNLP predict...
2020-07-21 19:04:29,549 - INFO - pytorch_pretrained_bert.modeling - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2020-07-21 19:04:30,074 - INFO - pytorch_transformers.modeling_bert - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2020-07-21 19:04:30,077 - INFO - pytorch_transformers.modeling_xlnet - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2020-07-21 19:04:30,459 - INFO - allennlp.common.registrable - instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
2020-07-21 19:04:30,460 - INFO - allennlp.common.registrable - instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
2020-07-21 19:04:30,461 - INFO - allennlp.common.registrable - instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
2020-07-21 19:04:30,461 - INFO - allennl

In [ ]:
!head summarunner_predictions.txt

американское аэрокосмическое агентство nasa огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. каждый проект из выбранных полуфиналистов получит $3 млн на проработку концепции. из четырех выбранных проектов nasa планирует отобрать две в 2021 году, чтобы профинансировать работу по их реализации в рамках программы discovery.
около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре бурятии. более 300 артистов из одного региона на главной сцене страны - похоже это рекорд россии. и все они были представлены в москве.
7 ноября в белоруссии прошли выборы членов совета республики — верхней палаты парламента. в столице и шести областях республики были избраны по восемь сенаторов. еще восемь членов совета назначит президент белоруссии александр лукашенко.
народная артистка рсфср надежда бабкина в интервью шоу «секрет на миллион» леры кудрявцевой 

## Pointer-Generator
* Original paper: https://arxiv.org/abs/1704.04368

In [ ]:
!wget https://www.dropbox.com/s/2yk25xaizevtqw3/gazeta_pgn_7kk_cov.tar.gz

--2020-07-21 19:08:30--  https://www.dropbox.com/s/2yk25xaizevtqw3/gazeta_pgn_7kk_cov.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/2yk25xaizevtqw3/gazeta_pgn_7kk_cov.tar.gz [following]
--2020-07-21 19:08:31--  https://www.dropbox.com/s/raw/2yk25xaizevtqw3/gazeta_pgn_7kk_cov.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc5b8a3de6a6215926e5e98ba3d.dl.dropboxusercontent.com/cd/0/inline/A78p9i5RxQs2-HD7h4oKA5LrTaxVqXQjgO0eytRw7OahEy1ZWaVS-og3g0SKlWKd-YV-9vl-SM-0OEn_D-8cb7HwXJVN87D9r674n9XqBxrqJzN_8RgpLICbDoXNw2a9jDw/file# [following]
--2020-07-21 19:08:31--  https://ucc5b8a3de6a6215926e5e98ba3d.dl.dropboxusercontent.com/cd/0/inline/A78p9i5RxQs2-HD7h4oKA5LrTaxVqXQjgO0eytRw7OahEy1ZWaVS-og3g0SKlWKd-YV-9vl-SM-

In [ ]:
!bash predict.sh -t gazeta_test.jsonl -m gazeta_pgn_7kk_cov.tar.gz -p subwords_summary \
    -L ru -T -D -P pgn_predictions.txt -M meteor-1.5/meteor-1.5.jar

Calling AllenNLP predict...
2020-07-21 19:09:08,932 - INFO - pytorch_pretrained_bert.modeling - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2020-07-21 19:09:09,462 - INFO - pytorch_transformers.modeling_bert - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2020-07-21 19:09:09,466 - INFO - pytorch_transformers.modeling_xlnet - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2020-07-21 19:09:09,860 - INFO - allennlp.common.registrable - instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
2020-07-21 19:09:09,861 - INFO - allennlp.common.registrable - instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
2020-07-21 19:09:09,862 - INFO - allennlp.common.registrable - instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
2020-07-21 19:09:09,862 - INFO - allennl

## mBART
* BART paper: https://arxiv.org/abs/1910.13461
* mBART paper: https://arxiv.org/abs/2001.08210

In [ ]:
import json
import torch
from transformers import MBartTokenizer, MBartForConditionalGeneration


def gen_batch(inputs, batch_size):
    batch_start = 0
    while batch_start < len(inputs):
        yield inputs[batch_start: batch_start + batch_size]
        batch_start += batch_size


def predict(
    model_name,
    test_file,
    max_source_tokens_count=600,
    max_target_tokens_count=160,
    use_cuda=True,
    batch_size=4
):
    inputs = []
    targets = []
    with open(test_file, "r") as r:
        for line in r:
            record = json.loads(line)
            inputs.append(record["text"])
            targets.append(record["summary"])

    inputs = inputs[:50]
    targets = targets[:50]

    tokenizer = MBartTokenizer.from_pretrained(model_name)
    device = torch.device("cuda:0") if use_cuda else torch.device("cpu")
    model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
    predictions = []
    for batch in gen_batch(inputs, batch_size):
        input_ids = tokenizer.prepare_seq2seq_batch(
            batch,
            src_lang="en_XX",
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_source_tokens_count
        )["input_ids"].to(device)
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_target_tokens_count + 2,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.extend(summaries)
    return inputs, predictions

inputs, predictions = predict("IlyaGusev/mbart_ru_sum_gazeta", "gazeta_test.jsonl")
print("Origin: ", inputs[7])
print("Summary: ", predictions[7])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Origin:  Самые известные российские пранкеры Владимир «Вован» Кузнецов и Алексей «Лексус» Столяров, на счету которых множество розыгрышей мировых политиков первого эшелона, позвонили члену конгресса США от Демократической партии Максин Уотерс от имени шведской школьницы, экологической активистки Греты Тунберг и ее отца Сванте. Звонок был сделан в рамках нового анимационного шоу пранкеров Stars save the Earth. Россияне рассказали американскому политику, что Грета якобы сейчас участвует в митинге в штате Северная Каролина «в поддержку экологии острова Чунга-Чанга». В этой связи «Тунберги» предложили Уотерс также поддержать экологию острова (несуществующего), чтобы ее речь затем якобы транслировать на мероприятии, передает РИА «Новости». «Я очень рада, что Грета с вами в Северной Каролине, где вы сосредоточены на защите очень важного острова Чунга-Чанга. Самое главное сейчас — убедиться, что остров защищен, что вы даете ему всю поддержку, которую вы можете дать», — заявила конгрессвумен С

The model was trained via [fairseq](https://github.com/pytorch/fairseq). 

Fairseq is a sequence modeling toolkit by the Facebook that allows researchers and developers to train custom models for sequence modeling (translation, summarization, language modeling and other text generation tasks).

Trained script is provided below

In [ ]:
# train sсript for mbart
https://github.com/IlyaGusev/summarus/blob/master/external/bart_scripts/train.sh

In [ ]:
!cd .. && git clone https://github.com/pytorch/fairseq && cd fairseq && git checkout a06083f && pip install --editable .

Cloning into 'fairseq'...
remote: Enumerating objects: 17659, done.
remote: Total 17659 (delta 0), reused 0 (delta 0), pack-reused 17659
Receiving objects: 100% (17659/17659), 8.05 MiB | 12.63 MiB/s, done.
Resolving deltas: 100% (13072/13072), done.
Note: checking out 'a06083f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a06083f8 Fix score reference when using translation_from_pretrained_bart task
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |██████████████

In [ ]:
!mkdir gazeta_mbart_model
!wget https://www.dropbox.com/s/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz -O gazeta_mbart_model/gazeta_mbart_checkpoint_600_160.tar.gz 
!wget https://www.dropbox.com/s/rqnwjuvp91vhni5/gazeta_data_mbart_600_160.tar.gz
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.CC25.tar.gz
!cd gazeta_mbart_model && tar -xzvf gazeta_mbart_checkpoint_600_160.tar.gz
!tar -xzvf gazeta_data_mbart_600_160.tar.gz
!tar -xzvf mbart.CC25.tar.gz
!rm gazeta_data_mbart_600_160.tar.gz
!rm mbart.CC25.tar.gz
!rm gazeta_mbart_model/gazeta_mbart_checkpoint_600_160.tar.gz

--2020-09-15 13:54:11--  https://www.dropbox.com/s/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz [following]
--2020-09-15 13:54:12--  https://www.dropbox.com/s/raw/b2auu9dhrm2wj0p/gazeta_mbart_checkpoint_600_160.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc82a02b203e014b96d04b34fdc9.dl.dropboxusercontent.com/cd/0/inline/A_ediQpJSPtj3UPKF__38_jo-bV2ptWE1qqAeLYiXSTf3LzF8niOkFATYkIsw7dGGpAqccbBS9t24UDg7aOT8aYLl0xIFiVyUBP-KC0GoFj5WbqqAUMMwcmuTNkD5SbLd6c/file# [following]
--2020-09-15 13:54:12--  https://uc82a02b203e014b96d04b34fdc9.dl.dropboxusercontent.com/cd/0/inline/A_ediQpJSPtj3UPKF__38_jo-bV2ptWE1

In [ ]:
# Not enough memory in Colab
!sh external/bart_scripts/predict.sh gazeta_mbart_model/checkpoint12.pt \
    data-bin ../fairseq \
    mbart.cc25/sentence.bpe.model mbart_predictions.txt 1

In [ ]:
!wget -nc https://www.dropbox.com/s/zccr34rtmb6r6oi/mbart_predictions.txt
!head -n 100 mbart_predictions.txt

File ‘mbart_predictions.txt’ already there; not retrieving.

2020-07-21 22:31:54 | INFO | fairseq_cli.generate | Namespace(all_gather_list_size=16384, beam=5, bpe='sentencepiece', checkpoint_suffix='', cpu=False, criterion='cross_entropy', data='data-bin', data_buffer_size=0, dataset_impl=None, decoding_format=None, diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='test', iter_decode_eos_penalty=0.0, iter_decode_force_max_iter=False, iter_decode_max_iter=10, iter_decode_with_beam=1, iter_decode_with_external_reranker=False, langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,n

In [ ]:
predictions = {}
with open("mbart_predictions.txt", "r") as r:
    for line in r:
        if not line.startswith("D-"):
            continue
        sample_id = int(line.split("\t")[0][2:])
        pred = line.split("\t")[-1]
        predictions[sample_id] = pred.strip()
with open("mbart_predictions_fixed.txt", "w") as w:
    for i in range(len(test_records)):
        w.write(predictions[i] + "\n")

In [ ]:
!python3 target_to_lines.py --archive-file gazeta_summarunner_3kk.tar.gz \
  --input-file gazeta_test.jsonl --output-file target.txt

100% 338604/338604 [00:00<00:00, 616119.63B/s]


In [ ]:
!python3 evaluate.py --predicted-path mbart_predictions_fixed.txt \
  --gold-path target.txt --language ru --tokenize-after --lower

-------------METRICS-------------
Count:	 5770
Ref:	 россия пригласила китай принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов — и пекин воспользуется этим для создания собственного атомного авианосца . по данным аналитиков , китайцы хотят спустить на воду авианосец , сопоставимый по размерам и характеристикам с американскими кораблями типа gerald r . ford .
Hyp:	 китай создает свой собственный атомный авианосец с использованием российских технических новшеств и ноу-хау в сфере ядерного оружия . в частности , москва пригласила пекин принять участие в тендере на строительство нового класса атомного ледокола , что обязательно потребует разработки реакторов .
BLEU:     	50.1
ROUGE-1-F:	32.1
ROUGE-2-F:	14.2
ROUGE-L-F:	27.9
Dup 1-grams:	15.2
Dup 2-grams:	1.2
Dup 3-grams:	0.3


### Pegasus 

Origin paper: https://arxiv.org/abs/1912.08777

Model: https://huggingface.co/google/pegasus-cnn_dailymail

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

class PegasusCheck():

    def __init__(self, model_name):
        self.torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = PegasusTokenizer.from_pretrained(model_name)
        self.model = PegasusForConditionalGeneration.from_pretrained(model_name).to(self.torch_device)
    
    def make_summary(self, src_text):
        batch = self.tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(self.torch_device)
        # batch = self.tokenizer.prepare_seq2seq_batch([body], truncation=True, padding='longest').to(self.torch_device)
        translated = self.model.generate(**batch)
        tgt_text = self.tokenizer.batch_decode(translated, skip_special_tokens=True)
        return tgt_text[0]

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
pgd = PegasusCheck("google/pegasus-cnn_dailymail") # can be also trained on xsum 'google/pegasus-xsum'

In [ ]:
summary = pgd.make_summary(test_records_en[1]["text"])
summary

'kurdish peshmerga fighters claimed to take control of sinjar the northern iraqi town that isis militants stormed this summer.<n>The advance is the latest in recent days by kurdish forces against isis which has been blamed for atrocities around the area.<n>Last summer isis slaughtered yazidis by the hundreds according to vian dakhil the only lawmaker representing the yazidis in iraqs parliament women in the group were treated like cattle.'

In [ ]:
scores = rouge.get_scores(summary, test_records_en[1]["text"])
print(scores)

[{'rouge-1': {'f': 0.29120878864343075, 'p': 0.9636363636363636, 'r': 0.1715210355987055}, 'rouge-2': {'f': 0.21558872102472715, 'p': 0.9420289855072463, 'r': 0.12172284644194757}, 'rouge-l': {'f': 0.17596283438321603, 'p': 0.9636363636363636, 'r': 0.1715210355987055}}]
